# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
pd.options.mode.chained_assignment = None  # default='warn'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
cities_df = pd.read_csv("../output_data/clean_city_data.csv")
cities_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,12.02,87,91,0.96,US,1663414655
1,Punta Arenas,-53.1500,-70.9167,4.01,75,75,5.66,CL,1663414655
2,Naze,28.3667,129.4833,27.51,89,100,15.69,JP,1663414656
3,Santa Fe,-31.6333,-60.7000,18.34,74,100,5.36,AR,1663414656
4,Hilo,19.7297,-155.0900,27.93,83,75,3.09,US,1663414657
...,...,...,...,...,...,...,...,...,...
552,Qui Nhon,13.7667,109.2333,27.14,81,48,1.16,VN,1663414849
553,Zaysan,47.4667,84.8667,13.34,37,12,2.40,KZ,1663414907
554,Westport,41.1415,-73.3579,18.19,77,100,3.60,US,1663414907
555,Pakxan,18.3778,103.6601,24.59,95,100,0.93,LA,1663414908


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Using the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]

In [22]:
# Creating and adding heatmap
map = gmaps.figure()
heat = gmaps.heatmap_layer(locations, weights=humidity, point_radius=4, max_intensity=100, dissipating=False)
map.add_layer(heat)

map

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Creating a DataFrame with only ideal weather
temp_df = cities_df[(cities_df["Max Temp"] > 21) & (cities_df["Max Temp"] < 27)]
wind_df = temp_df[temp_df["Wind Speed"] < 4.5]
weather_df = wind_df[wind_df["Cloudiness"] == 0]
weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Kapaa,22.0752,-159.3190,22.99,81,0,3.60,US,1663414658
54,Kahului,20.8947,-156.4700,24.36,73,0,1.54,US,1663414677
164,Safford,32.8340,-109.7076,21.73,55,0,0.00,US,1663414727
252,Koungou,-12.7336,45.2042,26.84,69,0,4.12,YT,1663414767
266,Yuci,37.6803,112.7319,21.86,53,0,1.00,CN,1663414774
412,Coachella,33.6803,-116.1739,26.26,51,0,3.09,US,1663414839
416,Seoul,37.5683,126.9778,25.78,83,0,2.06,KR,1663414780
436,Chickasha,35.0526,-97.9364,23.09,83,0,3.09,US,1663414852
516,Dickinson,29.4608,-95.0513,26.66,93,0,0.00,US,1663414889


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Storing weather_df into hotel_df variable
hotel_df = weather_df
# Adding a "Hotel Name" column
hotel_df["Hotel Name"] = ""
# Setting parameters
parameters = {"radius" : 5000, "types" : "hotel", "keyword" : "hotels", "key" : g_key}

# Obtaining the latitude and longitude of each city and using it to find hotels
for x, row in hotel_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["Latitude"]
    lng = row["Longitude"]
    parameters["location"] = str(lat) + ", " + str(lng)
    hotel = requests.get(url, params=parameters).json()
    try:
        hotel_df.loc[x, "Hotel Name"] = hotel["results"][0]["name"]
    except:
        hotel_df.loc[x, "Hotel Name"] = "N/A"
        print("No hotels found near " + str(row["City"]))
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,Kapaa,22.0752,-159.3190,22.99,81,0,3.60,US,1663414658,The ISO
54,Kahului,20.8947,-156.4700,24.36,73,0,1.54,US,1663414677,Maui Beach Hotel
164,Safford,32.8340,-109.7076,21.73,55,0,0.00,US,1663414727,SpringHill Suites by Marriott Thatcher
252,Koungou,-12.7336,45.2042,26.84,69,0,4.12,YT,1663414767,Villa Maora
266,Yuci,37.6803,112.7319,21.86,53,0,1.00,CN,1663414774,万豪美悦酒店
412,Coachella,33.6803,-116.1739,26.26,51,0,3.09,US,1663414839,Holiday Inn Express & Suites Indio - Coachella...
416,Seoul,37.5683,126.9778,25.78,83,0,2.06,KR,1663414780,Solaria Nishitetsu Hotel Seoul
436,Chickasha,35.0526,-97.9364,23.09,83,0,3.09,US,1663414852,"Holiday Inn Express & Suites Chickasha, an IHG..."
516,Dickinson,29.4608,-95.0513,26.66,93,0,0.00,US,1663414889,Hampton Inn & Suites Houston/League City


In [25]:
# Drop rows where no hotels could be found
hotel_df = hotel_df[hotel_df["Hotel Name"]!="N/A"]
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,Kapaa,22.0752,-159.3190,22.99,81,0,3.60,US,1663414658,The ISO
54,Kahului,20.8947,-156.4700,24.36,73,0,1.54,US,1663414677,Maui Beach Hotel
164,Safford,32.8340,-109.7076,21.73,55,0,0.00,US,1663414727,SpringHill Suites by Marriott Thatcher
252,Koungou,-12.7336,45.2042,26.84,69,0,4.12,YT,1663414767,Villa Maora
266,Yuci,37.6803,112.7319,21.86,53,0,1.00,CN,1663414774,万豪美悦酒店
412,Coachella,33.6803,-116.1739,26.26,51,0,3.09,US,1663414839,Holiday Inn Express & Suites Indio - Coachella...
416,Seoul,37.5683,126.9778,25.78,83,0,2.06,KR,1663414780,Solaria Nishitetsu Hotel Seoul
436,Chickasha,35.0526,-97.9364,23.09,83,0,3.09,US,1663414852,"Holiday Inn Express & Suites Chickasha, an IHG..."
516,Dickinson,29.4608,-95.0513,26.66,93,0,0.00,US,1663414889,Hampton Inn & Suites Houston/League City


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
map.add_layer(markers)

map

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>